In [ ]:
import sys
sys.path.append("../")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import os,sys
from pathlib import Path
import json 
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

#General ML 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, silhouette_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from itertools import combinations

#In-house Module Imports
from config import Configuration 
from datasets import EclipseSampledDataset, VoltaSampledDataset
from utils import *

In [ ]:
user = "aksar"
logging.warning(f'Are you sure that you are: {user}?')

In [ ]:
#Update these if you are not the desired user
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/unseen_experiments'
SYSTEM = 'volta'
EXP_NAME = 'tpds_experiments'
CV_INDEX = 0
FEATURE_SELECTION = False
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function
MODEL_CONFIG = 'experiment_3' #rf_tuncer or rf_tuncer_worst_case
logging.warning('Results will be generated in {}, double check please!'.format(MODEL_CONFIG))

In [ ]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': MODEL_CONFIG,
                     })

with open(str(conf['experiment_dir']) + '/anom_dict.json') as f:
    ANOM_DICT = json.load(f)
with open(str(conf['experiment_dir']) + '/app_dict.json') as f:
    APP_DICT = json.load(f)    
    
APP_REVERSE_DICT = {}
for app_name, app_encoding in APP_DICT.items():
    APP_REVERSE_DICT[app_encoding] = app_name    

ANOM_REVERSE_DICT = {}
for anom_name, anom_encoding in ANOM_DICT.items():
    ANOM_REVERSE_DICT[anom_encoding] = anom_name        

In [ ]:
if SYSTEM == 'eclipse':
    
    eclipseDataset = EclipseSampledDataset(conf)
    train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(scaler=SCALER,borghesi=False,tsfresh = True)

elif SYSTEM == 'volta':
    voltaDataset = VoltaSampledDataset(conf)
    train_data, train_label, test_data, test_label = voltaDataset.load_dataset(scaler=SCALER,borghesi=False)
    
assert list(train_data.index) == list(train_label.index) #check the order of the labels     
assert list(test_data.index) == list(test_label.index) #check the order of the labels    

if FEATURE_SELECTION:
    selected_features = pd.read_csv(conf['experiment_dir'] / 'selected_features.csv')
    train_data = train_data[list(selected_features['0'].values)]
    test_data = test_data[list(selected_features['0'].values)]
    
train_label['anom_names'] = train_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
train_label['app_names']=train_label['app'].apply(lambda x: APP_REVERSE_DICT[x])
test_label['anom_names'] = test_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
test_label['app_names']=test_label['app'].apply(lambda x: APP_REVERSE_DICT[x])

all_data = pd.concat([train_data, test_data])
all_data = all_data.dropna(axis=1, how='any')
all_label = pd.concat([train_label,test_label])

train_data = all_data.loc[train_label.index]
test_data = all_data.loc[test_label.index]
    
logging.info("Train data shape %s",train_data.shape)
logging.info("Train label shape %s",train_label.shape)
logging.info("Test data shape %s",test_data.shape)  
logging.info("Test label shape %s",test_label.shape)

logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())        

## Unknown App Experiments

In [ ]:
SCALER = 'MinMax'

In [ ]:
nas_apps = ['lu','sp','ft','bt','cg','mg']
mantevo_apps = ['miniMD','CoMD','miniGhost','miniAMR']
other_apps = ['kripke']

all_app_names = list(APP_DICT.keys())

assert set(nas_apps + mantevo_apps + other_apps) == set(all_app_names)


cluster_one = ['ft','cg','mg'] #126
cluster_two = ['miniAMR','lu','miniGhost'] #126
cluster_three = ['sp','bt','miniMD','kripke','CoMD'] #max 6 

In [ ]:
all_test_app_groups = []
all_test_app_sets = []
for cluster in [cluster_one,cluster_two,cluster_three]:    
    all_test_app_groups.append(cluster)
    all_test_app_sets.append(set(cluster))

In [ ]:
#This doesn't hold the unique combinations, however it should have the total number
train_app_combinations = []

#90 different test_apps combination
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    #logging.info("Test apps: %s", test_apps)
    #logging.info("All Train apps: %s", train_apps_all)
    
    num_train_unknowns = [2,4,6]

    #126 or 31 different train_apps combination
    for num_train_unknown in num_train_unknowns:
        for train_apps in combinations(train_apps_all, num_train_unknown):
            train_app_combinations.append(train_apps)

In [ ]:
"""
This code blocks read the processed filenames so far and creates a dict using (train_apps,test_apps) as a key
It can also detect cases which are processed multiple times with different names
"""

not_covered_combinations = []
unique_counter_dict = {}
extra_filenames = []
delete_extra_files = False

for file in listdir(conf['results_dir']):
    
    if "result_dict" in file:

        filename = file.split('.')[0]
        splitted_filename = file.split('#')    

        train_apps = set(splitted_filename[0].split(':')[1].split("-"))
        test_apps = set(splitted_filename[1].split(':')[1].split("-"))
        
        if not (frozenset(train_apps),frozenset(test_apps)) in unique_counter_dict:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] = 1
        else:
            unique_counter_dict[(frozenset(train_apps),frozenset(test_apps))] += 1
            extra_filenames.append(file)
                    
        if (not test_apps in all_test_app_sets) and (not train_apps in train_app_combinations):
            not_covered_combinations.append([train_apps,test_apps])
                        
sorted_unique_counter_dict = dict(sorted(unique_counter_dict.items(), key=lambda x: x[1], reverse=True))            

if delete_extra_files:
    
    for filename in extra_filenames:

        generic_filename = "#".join(filename.split("#")[:-1])

        os.remove(Path(conf['results_dir']) / filename)
        os.remove(Path(conf['results_dir']) / (generic_filename + "#alert_dict.json"))    

In [ ]:
counter = 0
for test_apps in all_test_app_groups:
    
    train_apps_all = list(set(all_app_names) - set(test_apps))
    logging.info("Test apps: %s", test_apps)
    logging.info("All Train apps: %s", train_apps_all)
    
    test_apps_label = all_label[all_label['app_names'].isin(test_apps)]
    assert set(test_apps_label['app_names'].unique()) == set(test_apps)
    
    test_apps_data = all_data.loc[test_apps_label.index]
    assert list(test_apps_data.index) == list(test_apps_label.index) #check the order of the labels       
    
    num_train_unknowns = [2,4,6]
                
    for num_train_unknown in num_train_unknowns:    
        
        for train_apps_selected in combinations(train_apps_all, num_train_unknown):        
            
            train_apps_selected = list(train_apps_selected)
            logging.info("Selected train apps: %s", train_apps_selected)
                        
            if not (frozenset(train_apps_selected),frozenset(test_apps)) in unique_counter_dict:                
            
                train_apps_label = all_label[all_label['app_names'].isin(train_apps_selected)]
                train_apps_data = all_data.loc[train_apps_label.index]
                assert list(train_apps_data.index) == list(train_apps_label.index) #check the order of the labels 

                if SCALER == 'MinMax':

                    scaler = MinMaxScaler()    

                elif SCALER == 'Standard':

                    scaler = StandardScaler()
                    
                clf = RandomForestClassifier(random_state=0)
                
                pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])  
                
                fit_clf = pipeline.fit(train_apps_data, train_apps_label['anom'])

                pred_test = pipeline.predict(test_apps_data)                
                logging.info("\n %s",classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT))
                
                result_filename = "train:" + "-".join(train_apps_selected) + "#test:" + "-".join(test_apps) \
                            + "#model:" + clf.__class__.__name__ + "#cv:" + str(CV_INDEX)                    
                logging.info(result_filename)
                result_dict = classification_report(test_apps_label['anom'],pred_test, target_names=ANOM_DICT,output_dict=True)

                with open(Path(conf['results_dir']) / (result_filename + "#result_dict.json"), "w") as fp:
                    json.dump(result_dict, fp)

                FAR_AMR_Calculate(true_label=test_apps_label['anom'],pred_label=pred_test,
                                  result_dir=conf['results_dir'],
                                  save_name=result_filename,save=True)                                   
            else: 
                counter += 1
                logging.info("File exists")

### 5-fold CV Validation

In [ ]:
SCALER = 'MinMax'

if SCALER == 'MinMax':
    
    scaler = MinMaxScaler()    

elif SCALER == 'Standard':
    
    scaler = StandardScaler()

clf = RandomForestClassifier(random_state=0)

pipeline = Pipeline([('transformer', scaler), ('estimator', clf)])

In [ ]:
scoring = ['precision_macro', 'recall_macro', 'f1_macro', 'f1_weighted']
skf = StratifiedKFold(n_splits=5)

scores = cross_validate(pipeline, all_data, all_label['anom'].values,                         
                        cv=skf, 
                        scoring=scoring)

In [ ]:
scores